# Assignment 9: Bayesian Analysis

### Conditional Probability 

Q.1 Explain in few terms what is Naive Bayes. What is it considered Naive?

The Naive Bayesian classifier is based on Bayes’ theorem with the independence assumptions between predictors. 
A Naive Bayesian model is easy to build, with no complicated iterative parameter estimation which makes it particularly useful for very large datasets. Despite its simplicity, the Naive Bayesian classifier often does surprisingly well and is widely used because it often outperforms more sophisticated classification methods. 

Naive Bayes (NB) is 'naive' because it makes the assumption that features of a measurement are independent of each other. This is naive because it is (almost) never true.

In [57]:
# Imports
import matplotlib.pyplot as plt
import warnings
%matplotlib inline
warnings.filterwarnings("ignore")
import pandas as pd
from sklearn import (
    ensemble,
    model_selection,    
    preprocessing,
    tree,
)
from sklearn.metrics import (
    auc,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
)

import pandas_profiling

from sklearn.model_selection import train_test_split

data = pd.read_csv('shingles.csv')
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1841 entries, 0 to 1840
Data columns (total 6 columns):
Rash                       1841 non-null object
SwollenLymphNode           1841 non-null object
Chills                     1841 non-null object
PolymeraseChainReaction    1841 non-null object
VZVAntibodyTest            1841 non-null object
Blisters                   1841 non-null object
dtypes: object(6)
memory usage: 86.4+ KB
None


,Rash,SwollenLymphNode,Chills,PolymeraseChainReaction,VZVAntibodyTest,Blisters
0,no,no,no,no,pos,no
1,yes,no,no,no,neg,no
2,no,no,no,no,neg,no
3,no,no,no,no,neg,no
4,no,no,no,no,neg,no


Q.2. Does this data contain any missing values?

In [58]:
data.isnull().sum() #checking the dataset for NaN values .... NaN values in two columns
# data is not containing missing values.

Rash                       0
SwollenLymphNode           0
Chills                     0
PolymeraseChainReaction    0
VZVAntibodyTest            0
Blisters                   0
dtype: int64

Q.3. Split the data into 70/30 train test

In [59]:
from sklearn.model_selection import train_test_split

# converting column to "0" and "1" 
def boolean_to_numeric(x):
    # return 1 if column is true, 0 otherwise
    if x == 'yes':
        return 1
    else:
        return 0

# converting column to "0" and "1" 
def AntibodyTest_to_numeric(x):
    # return 1 if antibodytest is positive, 0 otherwise
    if x == 'pos':
        return 1
    else:
        return 0
    
# apply the function to this column
data['Rash'] = data['Rash'].apply(boolean_to_numeric)
data['SwollenLymphNode'] = data['SwollenLymphNode'].apply(boolean_to_numeric)
data['Chills'] = data['Chills'].apply(boolean_to_numeric)
data['PolymeraseChainReaction'] = data['PolymeraseChainReaction'].apply(boolean_to_numeric)
data['Blisters'] = data['Blisters'].apply(boolean_to_numeric)
data['VZVAntibodyTest'] = data['VZVAntibodyTest'].apply(AntibodyTest_to_numeric)

data.head(10)

X = data.loc[:, data.columns != 'Rash']
y = data.loc[:, data.columns == 'Rash']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Q.4. Train a Gaussian Naive Bayes model, a Multinomial Naive Bayes and a Bernoulli Naive Bayes on the dataset to predict Rash. Compute the accuracy for each. Explain your results. 

In [60]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Bernoulli Naive bayes based model
BernNB = BernoulliNB(binarize=True)
BernNB.fit(X_train, y_train)
print(BernNB)

y_pred = BernNB.predict(X_test)
print(accuracy_score(y_test, y_pred))

# Multinomial Naive Bayes based model
MultiNB = MultinomialNB()
MultiNB.fit(X_train, y_train)
print(MultiNB)

y_pred = MultiNB.predict(X_test)
print(accuracy_score(y_test, y_pred))

# Gaussian Naive based model
GausNB = GaussianNB()
GausNB.fit(X_train, y_train)
print(GausNB)

y_pred = GausNB.predict(X_test)
print(accuracy_score(y_test, y_pred))

BernoulliNB(alpha=1.0, binarize=True, class_prior=None, fit_prior=True)
0.48282097649186256
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
0.5334538878842676
GaussianNB(priors=None, var_smoothing=1e-09)
0.5135623869801085


Q.5. Utilizing Pipeline and GridSearchCV, use 5 different alpha values to train a Bernoulli Naive Bayes and Multinomial Naive Bayes on the dataset. Print out the accuracy for each, and explain your results.

In [61]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import numpy as np

# ------ MulitNomial Naive Bayes results ----------

grid_params = {
  'mnb__alpha': (1000, 100, 1, 0.1, 0.001),
  'mnb__fit_prior': [True, False]
}

mnb_pipeline = Pipeline([
    ('mnb', MultinomialNB())
])

clf = GridSearchCV(mnb_pipeline, grid_params, scoring='accuracy', return_train_score=True)
clf.fit(X_train, y_train)
print("MulitNomial Naive Bayes results")
print("Best Score: ", clf.best_score_)
print("Best Params: ", clf.best_params_)

# ------ Bernoulli Naive Bayes results ----------

grid_params = {
  'bnl__alpha': (1000, 100, 1, 0.1, 0.001)
}

bnl_pipeline = Pipeline([
    ('bnl', BernoulliNB())
])

bnl = GridSearchCV(bnl_pipeline, grid_params, scoring='accuracy', return_train_score=True)
bnl.fit(X_train, y_train)
print("\nBernoulli Naive Bayes results")
print("Best Score: ", bnl.best_score_)
print("Best Params: ", bnl.best_params_)

# Tried out both Bernoulli and MulitNomial naive bayes on different alpha values and figured out the best possible alpha values
# in the set of 5. Multinomial has better accuracy over Bernoulli based naive bayes.

MulitNomial Naive Bayes results
Best Score:  0.5822981366459627
Best Params:  {'mnb__alpha': 1, 'mnb__fit_prior': True}

Bernoulli Naive Bayes results
Best Score:  0.5714285714285714
Best Params:  {'bnl__alpha': 100}


## Inference in Bayesian networks

Q.6. Create a new text cell in your Notebook: Complete a 50-100 word summary 
    (or short description of your thinking in applying this week's learning to the solution) 
     of your experience in this assignment. Include:
                                                                      
What was your incoming experience with this model, if any?
what steps you took, what obstacles you encountered.
how you link this exercise to real-world, machine learning problem-solving. (What steps were missing? What else do you need to learn?)
This summary allows your instructor to know how you are doing and allot points for your effort in thinking and planning, and making connections to real-world work.


It was interesting to learn the use of different types of Naive Based models. We could also learn the usage of pipeline and GridsearchCV to figure out the best possible configuration to run the models.
As explained during lesson, Bayes based inference models helps us in re-evaluating probability of certain evidence taking another event into consideration. 
Even though I gathered an overview of the lesson, I need to go through the basics of the lessob in some more depth.